# Spark NoSQL Database Workloads
Tested only with __postgresql__

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


## Instantiate Classes

In [15]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('rezaware/')[0])
from rezaware.modules.etl.loader import sparkNoSQL as db
from rezaware.modules.etl.loader import propAttr as attr

''' restart initiate classes '''
if debug:
    import importlib
    db = importlib.reload(db)
    attr = importlib.reload(attr)

__desc__ = "read and write documents from and to nosql database"
clsNoSQL = db.dataWorkLoads(
    desc=__desc__,
    db_type = 'mongodb',
    db_driver=None,
    db_hostIP=None,
    db_port = None,
    db_name = "tip-top-daily-assets",
    db_user = None,
    db_pswd = None,
    db_auth_source = "tip-top-daily-assets",
    db_auth_mechanism=None,
)
print("\n%s class initialization and load complete!" % __desc__)

All functional SPARKNOSQL-libraries in LOADER-package of ETL-module imported successfully!
All functional PROPATTR-libraries in LOADER-package of ETL-module imported successfully!
propAttr Class initialization complete
propAttr Class initialization complete

read and write documents from and to nosql database class initialization and load complete!


## Load data from DB using SQL query

In [26]:
clsNoSQL.read_documents(
    as_type = "DICT",
    db_name = "",
    db_coll = ['prime_pick.2022-05-01.top.23'],
    doc_find= {},
)

[{'_id': ObjectId('6676667e01c029b1493a9dcb'),
  'mcap.db.fk': 463103,
  'uuid': '65427b27810f8dddb9b2ad47',
  'asset.name': 'bitcoin',
  'asset.mcap.date': datetime.datetime(2022, 5, 1, 0, 0),
  'asset.mcap.value': 719611115951.295,
  'asset.price.date': datetime.datetime(2022, 5, 1, 0, 0),
  'asset.price.value': 37820.6117652921,
  'index.cov': 0.00011500720456650246,
  'audit.mod.by': 'NUWAN',
  'audit.mod.dt': '2024-06-22T13:51:57',
  'audit.mod.proc': 'propAttr-propAttr function <write_asset_picks_to_db>'},
 {'_id': ObjectId('6676667e01c029b1493a9dcc'),
  'mcap.db.fk': 567865,
  'uuid': '65428fac810f8dddb9b37adc',
  'asset.name': 'leo_token',
  'asset.mcap.date': datetime.datetime(2022, 5, 1, 0, 0),
  'asset.mcap.value': 5313723931.31921,
  'asset.price.date': datetime.datetime(2022, 5, 1, 0, 0),
  'asset.price.value': 5.68832875327235,
  'index.cov': -0.00012208932716995075,
  'audit.mod.by': 'NUWAN',
  'audit.mod.dt': '2024-06-22T13:51:57',
  'audit.mod.proc': 'propAttr-propAttr

In [24]:
len(clsNoSQL.documents)

23

## List database tables

## Replace modify attributes with Nulls
* Set modify_proc & modify_by values to Null
* Drop the modify_dt column

In [7]:
from datetime import date, datetime, timedelta
from pyspark.sql import functions as F

_upsert_sdf = hotels_sdf_ \
                    .filter(F.col('ref_pk').isin([23])) \
                    .withColumn('deactivate_dt',F.lit(datetime.now())) \
#                     .withColumn('modified_proc',F.lit(None))
# _upsert_sdf = _upsert_sdf.drop(F.col('modified_dt'))
_upsert_sdf.show(n=3,vertical=True)

-RECORD 0-----------------------------
 ref_pk        | 23                   
 entity        | hotel_group          
 category      | hotel group category 
 value         | Urlaubsbauernhof     
 code          | UB                   
 description   | null                 
 source_uuid   | 8619052              
 data_source   | https://teams.mic... 
 data_owner    | Delta Check          
 created_dt    | 2023-09-08 14:33:... 
 created_by    | rezaware             
 created_proc  | rezaware_io_impor... 
 modified_dt   | null                 
 modified_by   | null                 
 modified_proc | null                 
 deactivate_dt | 2023-09-08 16:53:... 



## Upsert table to verify Modify values are auto added]
* Load data again to verify that modify_dt is added and modify_by & modify_proc have values

In [9]:
_db_name ='property'
_tbl_name='util_refer'
_pk = ['ref_pk']
_cols_not_for_update = ['created_dt','created_by','created_proc']
_options={
    "BATCHSIZE":1000,   # batch size to partition the dtaframe
    "PARTITIONS":1,    # number of parallel clusters to run
    "OMITCOLS":_cols_not_for_update,    # columns to be excluded from update
    
}
_records=clsSDB.upsert_sdf_to_table(
    save_sdf=_upsert_sdf,
    db_name =_db_name,
    db_table=_tbl_name,
    unique_keys=_pk,
    **_options,
)

print("Upserted %d records" % _records)

Wait a moment, writing data to postgresql property database ...
 INSERT INTO curated.util_refer (ref_pk, entity, category, value, code, description, source_uuid, data_source, data_owner, created_dt, created_by, created_proc, modified_dt, modified_by, modified_proc, deactivate_dt) VALUES %s  ON CONFLICT (ref_pk) DO UPDATE SET (entity, category, value, code, description, source_uuid, data_source, data_owner, modified_dt, modified_by, modified_proc, deactivate_dt) = (EXCLUDED.entity, EXCLUDED.category, EXCLUDED.value, EXCLUDED.code, EXCLUDED.description, EXCLUDED.source_uuid, EXCLUDED.data_source, EXCLUDED.data_owner, EXCLUDED.modified_dt, EXCLUDED.modified_by, EXCLUDED.modified_proc, EXCLUDED.deactivate_dt) ;


dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>
dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>


Upserted 1 records


dataWorkLoads @staticmethod <batch_and_upsert> PSQL connection set with <class 'psycopg2.extensions.cursor'> and connection <connection object at 0x7f73e7976180; dsn: 'user=rezaware password=xxx dbname=property host=127.0.0.1 port=5432', closed: 0>


## Get table next PK

In [16]:
next_pk = clsSDB.get_table_pk_nextval(
    tbl_name='hotel_detail',
    pk_attr ='hotel_pk',
)

next_pk.show()

+-------+
|nextval|
+-------+
|      7|
+-------+



## Set table PK index with new value

In [37]:
set_pk_val_ = clsSDB.set_table_pk_lastval(
    tbl_name= "prop_grp",
    pk_attr = "prop_grp_pk",
    set_val = 1
)
set_pk_val_

1